# TP4 : Classification d'Images avec SVM et Random Forest

## Activité 1 : Entraînement du modèle SVM

Extraire les descripteurs SIFT, créer un vocabulaire visuel avec K-means, calculer les histogrammes et entraîner un modèle SVM.

In [16]:
import numpy as np
import os
from skimage.feature import SIFT
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import joblib

# Étape 1 : Lister les images et attribuer des identifiants de classe
train_path = 'images/dataset/train'
training_names = os.listdir(train_path)
image_paths = []
image_classes = []
class_id = 0

def imglist(path):
    # Retourne la liste des chemins d'images avec extensions .jpg ou .png
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith(('.jpg', '.png'))]

for training_name in training_names:
    dir_path = os.path.join(train_path, training_name)
    if os.path.isdir(dir_path):
        class_path = imglist(dir_path)
        image_paths += class_path
        image_classes += [class_id] * len(class_path)
        class_id += 1

print(f'Nombre total d\'images : {len(image_paths)}, Classes : {class_id}')

# Étape 2 : Extraire les descripteurs SIFT
descriptor_extractor = SIFT()
des_list = []

for image_path in image_paths:
    im = np.array(Image.open(image_path).convert('L').resize((128, 128)))
    try:
        descriptor_extractor.detect_and_extract(im)
        descriptors = descriptor_extractor.descriptors
        if descriptors is not None and len(descriptors) > 0:
            des_list.append((image_path, descriptors))
    except:
        print(f'Échec de l\'extraction des descripteurs pour {image_path}')

# Empiler tous les descripteurs
descriptors = des_list[0][1]
for _, descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))

print(f'Nombre total de descripteurs : {descriptors.shape}')

# Étape 3 : Créer un vocabulaire visuel avec K-means
k = 100  # Nombre initial de clusters
kmeans = KMeans(n_clusters=k, random_state=30)
kmeans.fit(descriptors)
codebook = kmeans.cluster_centers_

# Étape 4 : Calculer les histogrammes BoF
im_features = np.zeros((len(image_paths), k), 'float32')
for i in range(len(image_paths)):
    if i < len(des_list):
        words = kmeans.predict(des_list[i][1])
        for w in words:
            im_features[i][w] += 1

# Normaliser les caractéristiques
stdslr = StandardScaler().fit(im_features)
im_features = stdslr.transform(im_features)

# Étape 5 : Entraîner le SVM
clf = LinearSVC(max_iter=10000, random_state=30)
clf.fit(im_features, np.array(image_classes))

# Étape 6 : Sauvegarder le modèle
joblib.dump((clf, training_names, stdslr, k, codebook), 'bof_model.pkl', compress=3)
print('Modèle SVM entraîné et sauvegardé sous bof_model.pkl')

Nombre total d'images : 409, Classes : 3
Nombre total de descripteurs : (63549, 128)
Modèle SVM entraîné et sauvegardé sous bof_model.pkl


## Activité 2 : Prédiction de la classe d'une image

Nous allons charger le modèle sauvegardé, extraire les caractéristiques d'une image de test et prédire sa classe. Ensuite, nous ré-entraînerons le modèle avec k=50.

In [17]:
# Étape 1 : Charger le modèle sauvegardé
clf, classes_names, stdslr, k, codebook = joblib.load('bof_model.pkl')

# Étape 2 : Extraire les descripteurs SIFT de l'image de test
test_path = 'images/dataset/test/sea/sea_test-001.jpg'
im = np.array(Image.open(test_path).convert('L').resize((128, 128)))
descriptor_extractor.detect_and_extract(im)
test_descriptors = descriptor_extractor.descriptors

# Étape 3 : Calculer l'histogramme BoF pour l'image de test
test_features = np.zeros((1, k), 'float32')
if test_descriptors is not None:
    words = kmeans.predict(test_descriptors)
    for w in words:
        test_features[0][w] += 1

# Normaliser les caractéristiques de test
test_features = stdslr.transform(test_features)

# Étape 4 : Prédire la classe
prediction = clf.predict(test_features)
predicted_class = classes_names[prediction[0]]
print(f'Classe prédite pour l\'image de test : {predicted_class}')

# Étape 5 : Ré-entraîner avec k=50
k = 50
kmeans = KMeans(n_clusters=k, random_state=30)
kmeans.fit(descriptors)
codebook = kmeans.cluster_centers_

im_features = np.zeros((len(image_paths), k), 'float32')
for i in range(len(image_paths)):
    if i < len(des_list):
        words = kmeans.predict(des_list[i][1])
        for w in words:
            im_features[i][w] += 1

stdslr = StandardScaler().fit(im_features)
im_features = stdslr.transform(im_features)

clf = LinearSVC(max_iter=10000, random_state=30)
clf.fit(im_features, np.array(image_classes))

joblib.dump((clf, training_names, stdslr, k, codebook), 'bof_model_k50.pkl', compress=3)
print('Modèle SVM ré-entraîné avec k=50 et sauvegardé sous bof_model_k50.pkl')

Classe prédite pour l'image de test : sea
Modèle SVM ré-entraîné avec k=50 et sauvegardé sous bof_model_k50.pkl


## Activité 3 : Utiliser l'algorithme Random Forest

Nous allons entraîner un classificateur Random Forest avec les mêmes caractéristiques et comparer ses performances avec le SVM.

In [21]:
from sklearn.ensemble import RandomForestClassifier

# # Entraîner Random Forest
clf_rf = RandomForestClassifier(n_estimators=100, random_state=30)
clf_rf.fit(im_features, np.array(image_classes))

# # Sauvegarder le modèle Random Forest
joblib.dump((clf_rf, training_names, stdslr, k, codebook), 'bof_rf_model.pkl', compress=3)
print('Modèle Random Forest entraîné et sauvegardé sous bof_rf_model.pkl')

# # Prédire avec Random Forest
# Recalculate test features using k=50
test_features_k50 = np.zeros((1, k), 'float32')
if test_descriptors is not None:
	words_k50 = kmeans.predict(test_descriptors)
	for w in words_k50:
		test_features_k50[0][w] += 1

# Normalize the test features
test_features_k50 = stdslr.transform(test_features_k50)

# Predict with Random Forest
prediction_rf = clf_rf.predict(test_features_k50)
predicted_class_rf = classes_names[prediction_rf[0]]
print(f'Random Forest predicted class for test image: {predicted_class_rf}')

Modèle Random Forest entraîné et sauvegardé sous bof_rf_model.pkl
Random Forest predicted class for test image: sea


## Comparaison des résultats

**SVM vs Random Forest** :
- **SVM** : Efficace pour les données à haute dimension comme les histogrammes BoF, particulièrement avec des noyaux linéaires. Il performe mieux lorsque les classes sont bien séparées.
- **Random Forest** : Robuste au surajustement, gère bien les relations non linéaires et est moins sensible à la mise à l'échelle des caractéristiques. Il peut surpasser le SVM dans les cas de frontières de classe complexes.
- **Pertinence** : La pertinence des prédictions dépend du dataset. Si la classe prédite pour l'image de test correspond à la classe attendue (basée sur la similarité visuelle ou la vérité terrain), le modèle est considéré comme précis. Random Forest peut offrir une meilleure généralisation pour des datasets bruités ou variés, tandis que le SVM peut être plus précis pour des datasets plus petits et bien structurés.
- **Impact de k=50** : Réduire k de 100 à 50 diminue la taille du vecteur de caractéristiques, ce qui peut simplifier le modèle mais risque de perdre en pouvoir discriminant. L'effet dépend de la complexité du dataset.

Pour une comparaison quantitative, il serait idéal de diviser les données d'entraînement en ensembles d'entraînement/validation et de calculer la précision. Cependant, avec une seule image de test, la comparaison est qualitative (prédiction correcte/incorrecte).